# HW2

In [5]:
from gensim import downloader

vec_num = 50
GLOVE_PATH = f'glove-twitter-{vec_num}'
glove_twitter = downloader.load(GLOVE_PATH)


## Model 1: GloVe

In [6]:
import numpy as np

def open_and_split_file(file_path):
    with open(file_path) as f:
        lines = f.readlines()
        words = []
        tags = []
        for line in lines:
            try:
                word, tag = line.rstrip().split("\t")
                word = word.lower()
                if (word not in glove_twitter):
                    words.append(np.zeros(vec_num))
                else:
                    words.append(glove_twitter[word])
                tags.append(0 if tag == "O" else 1)

            except:
                continue
    return words, tags

In [7]:
train_words, train_labels = open_and_split_file("/home/student/hw2/NER_task_in_NLP/data/train.tagged")

In [8]:
dev_words, dev_labels = open_and_split_file("/home/student/hw2/NER_task_in_NLP/data/dev.tagged")

In [9]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=9)
knn.fit(train_words, train_labels)
y_pred = knn.predict(dev_words)

from sklearn.metrics import f1_score
f1_score(dev_labels, y_pred)

/anaconda/envs/azureml_py38/lib/python3.8/site-packages/sklearn/neighbors/_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


0.5860995850622407

## Model 2: Feed Forward

In [75]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import Dataset
from torch.optim import Adam
from sklearn.metrics import f1_score

In [67]:
class CustomDataset(Dataset):
    def __init__(self, path):
        # Create a path-to-label dictionary
        with open(path) as f:
            lines = f.readlines()
            words = []
            tags = []
            for line in lines:
                try:
                    word, tag = line.rstrip().split("\t")
                    word = word.lower()
                    if (word not in glove_twitter):
                        words.append(np.zeros(vec_num))
                    else:
                        words.append(glove_twitter[word])
                    tags.append(0 if tag == "O" else 1)

                except:
                    continue
        self.words = words
        self.tags = tags

    def __len__(self):
        return len(self.words)

    def __getitem__(self, index):
        word = self.words[index]
        tag = self.tags[index]
        word = torch.FloatTensor(word).squeeze()
        data = {"word": word, "labels": tag}
        return data

In [68]:
train_dataset = CustomDataset(path="/home/student/hw2/NER_task_in_NLP/data/train.tagged")
dev_dataset = CustomDataset(path="/home/student/hw2/NER_task_in_NLP/data/dev.tagged")
datasets = {"train": train_dataset, "dev": dev_dataset}

In [69]:
class FeedForwardNN(nn.Module):
    def __init__(self, vec_dim, num_classes, hidden_dim=100):
        super(FeedForwardNN, self).__init__()
        self.first_layer = nn.Linear(vec_dim, hidden_dim)
        self.second_layer = nn.Linear(hidden_dim, num_classes)
        self.activation = nn.ReLU()
        self.loss = nn.CrossEntropyLoss()

    def forward(self, word, labels=None):
        x = self.first_layer(word)
        x = self.activation(x)
        x = self.second_layer(x)
        if labels is None:
            return x, None
        loss = self.loss(x, labels)
        return x, loss

In [70]:
from sklearn.metrics import accuracy_score, f1_score
from torch.utils.data import DataLoader

def train(model, data_sets, optimizer, num_epochs: int, batch_size=16):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    data_loaders = {"train": DataLoader(data_sets["train"], batch_size=batch_size, shuffle=True),
                    "dev": DataLoader(data_sets["dev"], batch_size=batch_size, shuffle=False)}
    model.to(device)

    best_acc = 0.0

    for epoch in range(num_epochs):
        model.train()

        for batch in data_loaders['train']:
            batch_size = 0
            for k, v in batch.items():
                batch[k] = v.to(device)
                batch_size = v.shape[0]

            optimizer.zero_grad()
            _, loss = model(**batch)
            loss.backward()  # The important part
            optimizer.step()
                
    # Now use the dev dataset to evaluate the model.
    model.eval()
    predictions = torch.tensor([])
    tags = torch.tensor([])
    for batch in data_loaders['dev']:
        batch_size = 0
        for k, v in batch.items():
            batch[k] = v.to(device)
            batch_size = v.shape[0]

        optimizer.zero_grad()    
        with torch.no_grad():
            outputs, _ = model(**batch) 
            pred = outputs.argmax(dim=-1).clone().detach().cpu()
            predictions = torch.cat((predictions, pred), 0)
        tags = torch.cat((tags, (batch["labels"].clone().detach().cpu())), 0)

    score = f1_score(tags, predictions)
    print(f'F1 score: {score}')
                
    # with open('model.pkl', 'rb') as f:
    #     model = torch.load(f)
    return model

In [76]:
# Hyper-parameters that we can change:
# hidden_dim: the dimension of the hidden layer
# num_epochs: the number of epochs to train the model
# learning_rate: the learning rate of the optimizer (Adam) - find out more about it in the documentation.
model = FeedForwardNN(vec_num, 2, hidden_dim=int(vec_num*2))
optimizer = Adam(params=model.parameters())
model = train(model=model, data_sets=datasets, optimizer=optimizer, num_epochs=15)
print()

F1 score: 0.6101190476190477



## Model 3: LSTM

In [79]:
# Initialize the embedding matrix according to the glove from earlier.
embedding_mat = np.array(train_words)

In [ ]:
class MyNet(nn.Module):
    def __init__(self, embedding_mat, embedding_dim, hidden_dim=50, tag_dim=2):
        super().__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.word_embedding = nn.Embedding.from_pretrained(embedding_mat, freeze=True)
        self.lstm = nn.LSTM(self.embedding_dim, self.hidden_dim, batch_first=True)
        self.hidden2tag = nn.Sequential(nn.ReLU(),
                                        nn.Linear(self.hidden_dim, tag_dim))
        self.loss_fn = nn.NLLLoss()

    def forward(self, word, sentence_len, tags=None):
        embeds = self.word_embedding(word)  # TODO: Check if this is the correct way to use the embedding
        lstm_out, _ = self.lstm(embeds.view(1, -1, self.embedding_dim))
        tag_space = self.hidden2tag(lstm_out[range(len(sentence)), sentence_len - 1, :])
        tag_score = F.softmax(tag_space, dim=1)
        if tags is None:
            return tag_score, None
        loss = self.loss_fn(tag_score, tags)
        return tag_score, loss

## Old code

In [3]:
def create_glove_vector_old(word, data):
    temp_word = False
    if word not in glove_twitter.key_to_index:
        # print(f"{word} not an existing word in the model")
        # if you dont have this word - just skip it
        # return False
        
        if word.startswith("http"):
            # all links in train data are tagged O
            data.append(np.zeros(vec_dim))
            return
        
        try:
            # check if word is a number
            float(word)
            data.append(np.zeros(vec_dim))
            return
        except:
            pass
    
        # try stemming
        for i in range(min(len(word), 5)):
            if word[i:] in glove_twitter.key_to_index:
                temp_word = word[i:]
                break
            for j in range(1, min(len(word), 5)):
                if word[i:-j] in glove_twitter.key_to_index:
                    temp_word = word[i:-j]
                    break

    else:
        temp_word = word


    if temp_word:
        vec = glove_twitter[temp_word]
        data.append(vec)

    else:
        data.append(np.zeros(vec_dim))
        # print(word)
    

In [6]:
with open("/home/student/hw2/NER_task_in_NLP/data/train.tagged") as f:
    lines = f.readlines()
    words_count = 0
    prev_word = ''
    for i, line in enumerate(lines):
        try:
            word, tag = line.rstrip().split("\t")
            words_count += 1
            if  (word.startswith('http')):
                # print(tag)
                if tag != 'O': #and lines[i+1].rstrip().split('\t')[0] == ':':
                    print("yes")
                    print(word)
            #if line[i-1] != "\t\n" and "@" in word:#  \
            #and line[i-1].rstrip().split("\t")[0] == "RT" and line[i+1].rstrip().split("\t")[0] == ":":
                # print(line[i-1])
                #pass

        except:
            # print(line.rstrip().split("\t"))
            # # print(line)
            # print([line])
            # ! the line is blank representing \t\n
            # TODO thingy for end of sentence
            prev_word = ''
            continue
        prev_word = word